<a href="https://colab.research.google.com/github/MyPersonalTrainerr/Deep-Learning/blob/main/model/Inspect_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/google/mediapipe.git

Cloning into 'mediapipe'...
remote: Enumerating objects: 12113, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 12113 (delta 40), reused 52 (delta 19), pack-reused 11922
Receiving objects: 100% (12113/12113), 529.47 MiB | 11.88 MiB/s, done.
Resolving deltas: 100% (8470/8470), done.
Checking out files: 100% (2402/2402), done.


In [3]:
%cd mediapipe

/content/mediapipe


In [4]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import timeit

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="mediapipe/modules/pose_landmark/pose_landmark_lite.tflite")
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.allocate_tensors()

# input details
print(input_details)
print("=========================")

# output details
#  39*5, heatmap, presence, segmentation, visibility, 39*3
for output in output_details:
  print(output)
print("=========================")

[{'name': 'input_1', 'index': 0, 'shape': array([  1, 256, 256,   3], dtype=int32), 'shape_signature': array([  1, 256, 256,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
{'name': 'Identity', 'index': 310, 'shape': array([  1, 195], dtype=int32), 'shape_signature': array([  1, 195], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'Identity_1', 'index': 315, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([1, 1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantize

In [ ]:
# for image
# check the type of the input tensor
floating_model = input_details[0]['dtype'] == np.float32
print(f'is floating model: {floating_model}')

# NxHxWxC, H:1, W:2
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

#                         image processing


image = cv2.imread("football.jpg")
plt.imshow(image[:,:,::-1])
image_cropped = image[:image.shape[0], :image.shape[0], ::-1] # crop to avoid letterboxing step
plt.imshow(image_cropped)

img = cv2.resize(image_cropped, (256, 256))[np.newaxis, :, :, :]
img = (np.float32(img) - 0.0) / 255.0  # normalization (specified in tflite_converter_calculator, not in model card)
plt.imshow(img.squeeze())

#                           preciction

interpreter.set_tensor(input_details[0]['index'], img)
interpreter.invoke()
output_face_landmarks1 = interpreter.get_tensor(output_details[0]['index'])
output_face_landmarks2 = interpreter.get_tensor(output_details[1]['index'])
output_face_landmarks3 = interpreter.get_tensor(output_details[2]['index'])
output_face_landmarks4 = interpreter.get_tensor(output_details[3]['index'])
output_face_landmarks5 = interpreter.get_tensor(output_details[4]['index'])


#print(output_face_landmarks1.squeeze().shape)
output_face_landmarks = output_face_landmarks1.reshape(39,5)
#plt.imshow(image_cropped)
output_face_landmarks = output_face_landmarks[:33,:]
#print(output_face_landmarks)
face_landmark_x = output_face_landmarks[:, 0:1]
face_landmark_y = output_face_landmarks[:, 1:2]
#plt.plot(face_landmark_x/256*image_cropped.shape[0], (face_landmark_y/256)*image_cropped.shape[1], '.')


In [ ]:
# for video
vid = cv2.VideoCapture("Basketball jump shot.mp4")
while(True):
    try :
        # Capture the video frame
        # by frame
        ret, image_cropped = vid.read()
        image_cropped = cv2.cvtColor(image_cropped, cv2.COLOR_BGR2RGB)
        #image_cropped = image[:image.shape[0], :image.shape[0], ::-1]
        img = cv2.resize(image_cropped, (256, 256))[np.newaxis, :, :, :]
        img = (np.float32(img) - 0.0) / 255.0
        image_cropped = cv2.cvtColor(image_cropped, cv2.COLOR_RGB2BGR)
        interpreter.set_tensor(input_details[0]['index'], img)
        interpreter.invoke()
        output_face_landmarks = interpreter.get_tensor(output_details[0]['index'])
        #mask_0 = interpreter.get_tensor(output_details[0]['index']).reshape(39,5)
        mask = interpreter.get_tensor(output_details[1]['index'])
        mask_2 = interpreter.get_tensor(output_details[4]['index'])
        mask_2 =mask_2.reshape(39,3)        
        #print(mask)
        #print(output_face_landmarks.squeeze().shape)
        output_face_landmarks = output_face_landmarks.reshape(39,5)
        output_face_landmarks = output_face_landmarks[:33,:]
        face_landmark_x = output_face_landmarks[:, 0:1]/255*image_cropped.shape[1]
        face_landmark_y = output_face_landmarks[:, 1:2]/255*image_cropped.shape[0]
        z= output_face_landmarks[:, 2:3]
        visibilty = output_face_landmarks[:, 3:4]
        presence = output_face_landmarks[:, 4:5]
        if mask >0.98:        
            for i in range(len(face_landmark_x)):
                if presence[i]>5:
                    image_cropped = cv2.circle(image_cropped, (int((face_landmark_x[i])),int(face_landmark_y[i])), 3, (0,0,255), -1)
        cv2.imshow("window",image_cropped)
        if cv2.waitKey(25) & 0xFF == ord('q'):
          break
    except:
        break
  
    # Display the resulting frame
    
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

In [5]:
# get details for each layer
all_layers_details = interpreter.get_tensor_details() 
# for layer in all_layers_details:
#   print(layer) 

In [6]:
%cd ..

/content


In [8]:
import csv

csv_columns = ['name','index','shape', 'shape_signature', 'dtype', 'quantization', 'quantization_parameters', 'sparsity_parameters' ]
csv_file = "Layers.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for layer in all_layers_details:
            writer.writerow(layer)
except IOError:
    print("I/O error")